# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadGhGraphs import loadGhGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "data/2D_Truss_v1.3/gh/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 9.3843e-01   valLoss:9.6518e-01  time: 1.78e+00
epoch: 1   trainLoss: 6.9534e-01   valLoss:9.3511e-01  time: 1.50e+00
epoch: 2   trainLoss: 5.7308e-01   valLoss:9.3829e-01  time: 1.51e+00
epoch: 3   trainLoss: 4.5304e-01   valLoss:1.1721e+00  time: 1.50e+00
epoch: 4   trainLoss: 3.8352e-01   valLoss:3.1390e+00  time: 1.49e+00
epoch: 5   trainLoss: 3.4358e-01   valLoss:7.9249e+00  time: 1.53e+00
epoch: 6   trainLoss: 3.2030e-01   valLoss:6.6499e+00  time: 1.50e+00
epoch: 7   trainLoss: 2.7452e-01   valLoss:1.7711e+00  time: 1.49e+00
epoch: 8   trainLoss: 2.1389e-01   valLoss:4.6930e-01  time: 1.48e+00
epoch: 9   trainLoss: 2.5143e-01   valLoss:8.9877e-01  time: 1.50e+00
epoch: 10   trainLoss: 2.2361e-01   valLoss:8.9705e-01  time: 1.49e+00
epoch: 11   trainLoss: 2.0516e-01   valLoss:1.8518e+00  time: 1.48e+00
epoch: 12   trainLoss: 1.5505e-01   valLoss:1.2470e+00  time: 1.50e+00
epoch: 13   trainLoss: 1.4793e-01   valLoss:1.9098e+00  time: 1.48e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000016,0.002476,0.106621,0.879427,0.960887,0.796181,-1.649491
test,0.000021,0.002732,0.119209,0.785595,0.913011,0.775217,-0.965350


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)